We would make attempt to scrape the blocks of essays instead. 



In [43]:
import requests
from bs4 import BeautifulSoup

In [44]:
url = "https://safetyzone.iogp.org/HighPotentialEvents/detail.asp?id=2538"


In [45]:
response = requests.get(url)

In [46]:
soup1 = BeautifulSoup(response.content, 'html.parser')
#parse to find required data 
fields = soup1.find_all("table", class_="listTable")
data = fields[2].find_all("td")

haz = []
for vals in data:
    try:
        haz.append(vals.find("h5").text)
    except:
        continue
tabs = {haz[0]:haz[1], haz[3]:haz[4], haz[5]:haz[6],haz[7]:haz[8], haz[9]:haz[10], haz[11]:haz[12],haz[13]:haz[14]}
tabs


{'Year:': '2018',
 'Region:': 'Asia/Australasia',
 'Country:': 'AUSTRALIA',
 'Location:': 'Onshore',
 'Function:': 'Production',
 'Cause:': 'Other',
 'Activity:': 'Maintenance, inspection, testing'}

We can access all the data by iterating throught the incident numbers, we just need to make room for any errors and null values involved. 

In [47]:
#function to get url and content
def get_url(val):

  base_url = "https://safetyzone.iogp.org/HighPotentialEvents/detail.asp?id="
  url_main = base_url + str(val)
  response = requests.get(url_main)
  soup = BeautifulSoup(response.content, "html.parser")
  return soup

In [48]:
#Function to parse document and extract necessary fields

def parse_fields(content):
    fields = content.find_all("table", class_="listTable")
    data = fields[2].find_all("td")

    haz = []
    for vals in data:
        try:
            haz.append(vals.find("h5").text)
        except:
            continue
    tabs = {haz[0]:haz[1], haz[3]:haz[4], haz[5]:haz[6],haz[7]:haz[8], haz[9]:haz[10], haz[11]:haz[12],haz[13]:haz[14]}
    return tabs



In [49]:
#Now to import time and parse through 200 / / incidents randint(1298, 8662)
#we would create a function to do this 
#The incident numbers dont follow a pattern so we will get some errors when the page doesnt exist so we would break and continue for cases like this.

from random import seed
from random import randint


def scrape():
  tot = []
  seed(5) # seed random number generator
  
  for _ in range(200): # generate some integers
    val = randint(1067, 2519)

    try:
      tot.append(parse_fields(get_url(val)))
    except IndexError:
      continue
  tot
    
  return tot

near_miss = scrape()



In [50]:
near_miss[3]

{'Year:': '2017',
 'Region:': 'Africa',
 'Country:': 'CHAD',
 'Location:': 'Onshore',
 'Function:': 'Unspecified',
 'Cause:': 'Struck by',
 'Activity:': 'Transport - Land'}

In [54]:
#import necessary modules
import pandas as pd

near_miss1 = pd.DataFrame(near_miss, columns=["Activity:","Cause:","Country:", "Function:","Location:","Region:","Year:"])
near_miss1.head()

,Activity:,Cause:,Country:,Function:,Location:,Region:,Year:
0,"Construction, commissioning, decommissioning",Pressure release,NIGERIA,Construction,Onshore,Africa,2016
1,"Maintenance, inspection, testing",Explosions or burns,ANGOLA,Production,Offshore,Africa,2011
2,"Construction, commissioning, decommissioning",Pressure release,USA,Construction,Onshore,North America,2013
3,Transport - Land,Struck by,CHAD,Unspecified,Onshore,Africa,2017
4,"Maintenance, inspection, testing",Other,AUSTRALIA,Production,Onshore,Asia/Australasia,2017


In [63]:
near_miss1.describe()

,Activity:,Cause:,Country:,Function:,Location:,Region:,Year:
count,180,180,180,180,180,180,180
unique,13,11,47,5,2,7,11
top,"Drilling, workover, well services",Struck by,USA,Production,Onshore,Middle East,2012
freq,36,73,21,87,121,53,31


In [61]:
near_miss1.to_csv("near_miss.csv")